### df_all

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_testing = pd.read_csv('../Tbrain_Insurance/testing-set.csv')
df_all = pd.read_csv('../Tbrain_Insurance/df_all.csv')

In [3]:
df_all.shape

(351273, 272)

In [4]:
df = df_all

df_train = df[df['Next_Premium']!=100]
print(df_train.shape)
df_test = df[df['Next_Premium']==100]
print(df_test.shape)
print(df_train.shape[0]+df_test.shape[0])
print(df.shape[0])

(210763, 272)
(140510, 272)
351273
351273


In [5]:
df.isnull().sum().sum()

0

In [6]:
# ### Optional 
# # downsampling
# df_train = df_train.sample(10000).reset_index()
# df_train = df_train[df_train.columns[1:]]
# df_train.head()
# # df_test = df_test.sample (1400).reset_index()

In [8]:
df_train['Next_Premium'] = df_train.apply(lambda row: 0 if row['Next_Premium']==0 else 1, axis =1)

/Users/ching/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
y=df_train['Next_Premium']
print('continue',df_train[df_train['Next_Premium']==1].shape[0]/df_train.shape[0]*100)
print('cancelled',df_train[df_train['Next_Premium']==0].shape[0]/df_train.shape[0]*100)
y.head()

continue 78.0748992944682
cancelled 21.925100705531804


0    1
1    1
2    1
3    1
4    1
Name: Next_Premium, dtype: int64

In [13]:
train_ID = df_train['Policy_Number']
df_train = df_train.drop(['Policy_Number'], axis=1)
print(df_train.shape)
print(train_ID.shape)

(210763, 271)
(210763,)


In [14]:
df_test=df_test.drop(['Next_Premium'], axis=1)
print(df_test.shape)

(140510, 271)


In [15]:
test_ID=df_test['Policy_Number']
df_test=df_test.drop(['Policy_Number'], axis=1)
print(df_test.shape)
print(test_ID.shape)

(140510, 270)
(140510,)


## Reduce Dimension 

In [16]:
train = df_train.drop(['Next_Premium'], axis=1)
test = df_test
y_train = y

In [17]:
print(train.shape)
print(test.shape)
print(y.shape)

(210763, 270)
(140510, 270)
(210763,)


In [13]:
# import matplotlib.pyplot as plt
 
# ## standarlization
# from sklearn.preprocessing import StandardScaler
# stdsc = StandardScaler()
# train_std = stdsc.fit_transform(train)
# test_std = stdsc.fit_transform(test)
 
# X_std = train_std

#     #求共變異係數矩陣
# cov_mat = np.cov(X_std.T)
# print("共變異係數矩陣.shape=",cov_mat.shape)

 
# #求共變異係數矩陣 的特徵向量及特徵值
# eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
# print("特徵向量.shape=",eigen_vecs.shape)

 
# #計算解釋變異數比率 各特徵值/特徵值總和
# tot = sum(eigen_vals)
# var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
# cum_var_exp = np.cumsum(var_exp)

# # Make a list of (eigenvalue, eigenvector) tuples
# eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
#                for i in range(len(eigen_vals))]

# print("特徵值，特徵向量length：",len(eigen_pairs))
# eigen_pairs.sort(key=lambda k: k[0], reverse=True)

# #保留兩個最具影響力的特徵向量組成13x2 的投影矩陣W
# w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
#                eigen_pairs[1][1][:, np.newaxis]))

# train_pca = X_std.dot(w)
# ###################################

# X_std = test_std

#     #求共變異係數矩陣
# cov_mat = np.cov(X_std.T)
# print("共變異係數矩陣.shape=",cov_mat.shape)

 
# #求共變異係數矩陣 的特徵向量及特徵值
# eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
# print("特徵向量.shape=",eigen_vecs.shape)

 
# #計算解釋變異數比率 各特徵值/特徵值總和
# tot = sum(eigen_vals)
# var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
# cum_var_exp = np.cumsum(var_exp)

# # Make a list of (eigenvalue, eigenvector) tuples
# eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
#                for i in range(len(eigen_vals))]

# print("特徵值，特徵向量length：",len(eigen_pairs))
# eigen_pairs.sort(key=lambda k: k[0], reverse=True)

# #保留兩個最具影響力的特徵向量組成13x2 的投影矩陣W
# w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
#                eigen_pairs[1][1][:, np.newaxis]))

# test_pca = X_std.dot(w)

In [14]:
# train = train_pca
# test = test_pca
# y_train = y

### Modelling

In [18]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.exceptions import ConvergenceWarning
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


In [19]:
#Validation function
n_folds = 5

def mae_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    mae= (-cross_val_score(model, train.values, y_train.values, scoring="neg_mean_absolute_error", cv = kf))
    return(mae)

In [17]:
# Base models

In [20]:
# # LASSO Regression :
# # This model may be very sensitive to outliers. So we need to made it more robust on them. For that we use the sklearn's Robustscaler() method on pipeline :
# lasso = make_pipeline(RobustScaler(), Lasso(alpha = 2, random_state=1))
# print("lasso mae: \n", mae)
# print("lasso score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [21]:
# # Elastic Net Regression :
# # again made robust to outliers:
# #l1_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
# ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=2, l1_ratio=1, random_state=3))
# mae = mae_cv(ENet)
# print("ENet mae: \n", mae)
# print("ENet score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [22]:
# #Kernel Ridge Regression :
# Kernel_list = ['linear','rbf','sigmoid','poly','laplacian','cosine','chi2']
# alpha_list = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

# KRR = KernelRidge(alpha = 0.02, kernel = 'cosine', degree = 0) 
# # mae = mae_cv(KRR)
# # print("KRR mae: \n", mae)
# # print("KRR score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))


In [23]:
# # With huber loss that makes it robust to outliers:
# GBoost = GradientBoostingRegressor()
# # mae = mae_cv(GBoost)
# # print("GBoost mae: \n", mae)
# # print("GBoost score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [24]:
# # Averaged base models class
# class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models = models
        
#     # we define clones of the original models to fit the data in
#     def fit(self, X, y):
#         self.models_ = [clone(x) for x in self.models]
        
#         # Train cloned base models
#         for model in self.models_:
#             model.fit(X, y)

#         return self
    
#     #Now we do the predictions for cloned models and average them
#     def predict(self, X):
#         predictions = np.column_stack([
#             model.predict(X) for model in self.models_
#         ])
#         return np.mean(predictions, axis=1) 

In [25]:
## mae Score

In [26]:
# mae = mae_cv(lasso)
# mae_lasso = mae 
# print("lasso mae: \n", mae)
# print("lasso score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [27]:
# mae = mae_cv(ENet)
# mae_ENet = mae
# print("ENet mae: \n", mae)
# print("ENet score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [28]:
# mae = (mae_cv(KRR))
# mae_KRR = mae
# print("KRR mae: \n", mae)
# print("KRR score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [28]:
# mae = mae_cv(GBoost)
# mae_GBoost = mae
# print("GBoost mae: \n", mae)
# print("GBoost score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [29]:
# # Averaged base models score
# averaged_models = AveragingModels(models = (lasso, ENet, GBoost))

In [30]:
# mae = mae_cv(averaged_models)
# mae_Avg = mae
# print(" Averaged base models: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [31]:
# #Meta-Model
# #Stacking averaged Models Class:
# class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, base_models, meta_model, n_folds=5):
#         self.base_models = base_models
#         self.meta_model = meta_model
#         self.n_folds = n_folds
   
#     # We again fit the data on clones of the original models
#     def fit(self, X, y):
#         self.base_models_ = [list() for x in self.base_models]
#         self.meta_model_ = clone(self.meta_model)
#         kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
#         # Train cloned base models then create out-of-fold predictions
#         # that are needed to train the cloned meta-model
#         out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
#         for i, model in enumerate(self.base_models):
#             for train_index, holdout_index in kfold.split(X, y):
#                 instance = clone(model)
#                 self.base_models_[i].append(instance)
#                 instance.fit(X[train_index], y[train_index])
#                 y_pred = instance.predict(X[holdout_index])
#                 out_of_fold_predictions[holdout_index, i] = y_pred
                
#         # Now train the cloned  meta-model using the out-of-fold predictions as new feature
#         self.meta_model_.fit(out_of_fold_predictions, y)
#         return self
   
#     #Do the predictions of all base models on the test data and use the averaged predictions as 
#     #meta-features for the final prediction which is done by the meta-model
#     def predict(self, X):
#         meta_features = np.column_stack([
#             np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
#             for base_models in self.base_models_ ])
#         return self.meta_model_.predict(meta_features)

In [32]:
# def mae_(y, y_pred):
#     return mean_absolute_error(y,y_pred)

In [33]:
# # Stacking Averaged models Score

# stacked_averaged_models = StackingAveragedModels(base_models = (ENet, lasso, GBoost),  #ENet, lasso, KRR
#                                                  meta_model = GBoost) #GBoost

# mae = mae_cv(stacked_averaged_models)
# mae_stacked_avg = mae
# print("Stacking Averaged models score: {:.4f} ({:.4f})".format(mae.mean(), mae.std()))

In [38]:
# more models

In [34]:
lgb.LGBMClassifier?

In [35]:
# LGBM :
model_lgb = lgb.LGBMClassifier()
mae = mae_cv(model_lgb)
mae_lgb = mae
print("lgb mae: \n", mae)
print("lgb score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

lgb mae: 
 [ 0.17431737  0.18931037  0.19227576  0.21218448  0.17161701]
lgb score: 0.1879 (0.0146)



In [38]:
model = model_lgb  #

model.fit(train, y_train)
train_pred_lgb = model.predict(train)

y_pred_lgb = model.predict(test)  #

importances = model.feature_importances_
# std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(15):   # train.shape[1]
    print("%d. feature %d.: %str (%2f)" % (f + 1, indices[f], train.columns[indices[f]], importances[indices[f]]*100))


Feature ranking:
1. feature 73.: Distribution_Channeltr (25700.000000)
2. feature 96.: V_agetr (19000.000000)
3. feature 71.: Engine_Displacement_(Cubic_Centimeter)tr (13600.000000)
4. feature 110.: Insured_Amount3tr (13000.000000)
5. feature 72.: Replacement_cost_of_insured_vehicletr (11700.000000)
6. feature 101.: d_agetr (11300.000000)
7. feature 74.: Multiple_Products_with_TmNewa_(Yes_or_No?)tr (10100.000000)
8. feature 108.: Insured_Amount1tr (9900.000000)
9. feature 176.: Premiumtr (8800.000000)
10. feature 97.: i_agetr (8700.000000)
11. feature 78.: aassured_ziptr (8700.000000)
12. feature 260.: Premium_meantr (8200.000000)
13. feature 111.: Insured_Amount_overalltr (7800.000000)
14. feature 252.: Insured_Amount1_meantr (7100.000000)
15. feature 269.: Premium_Pred_Add_meantr (6800.000000)


In [39]:
# XGBoost :
model_xgb = xgb.XGBRegressor() #n_estimators=200
mae = mae_cv(model_xgb)
mae_xgb = mae 
print("Xgboost mae: \n", mae)
print("Xgboost score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

KeyboardInterrupt: 

In [ ]:
model = model_xgb  #

model.fit(train, y_train)
train_pred_xgb = model.predict(train)  #

y_pred_xgb = model.predict(test)  #

importances = model.feature_importances_
# std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(7):   # train.shape[1]
    print("%d. feature %d.: %str (%2f)" % (f + 1, indices[f], train.columns[indices[f]], importances[indices[f]]*100))


In [ ]:
# RF :
model_rf = RandomForestRegressor(n_estimators=100)  # default n_estimators = 10
mae = mae_cv(model_rf)
mae_rf = mae
print("RandomForest mae: \n", mae)
print("\nRandomForest score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [51]:
# mae_rf on train: 801.865598
# Feature ranking:
# 1. feature 166.: Premiumtr (29.192158)
# 2. feature 167.: Premium_Predtr (26.174226)
# 3. feature 258.: Premium_Pred_Addtr (8.405166)
# 4. feature 72.: Replacement_cost_of_insured_vehicletr (5.817675)
# 5. feature 71.: Engine_Displacement_(Cubic_Centimeter)tr (1.952259)
# 6. feature 242.: Insured_Amount1_meantr (1.222075)
# 7. feature 78.: aassured_ziptr (1.193084)

In [52]:
# Final Training and Prediction: 
stacked_averaged_models.fit(train.values, y_train.values)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = stacked_averaged_models.predict(test.values)

print(mae_(y_train, stacked_train_pred))

2002.98356142


In [53]:
l = 0.09
x = 0.1
r = 0.11
s = 1- l - x - r
ensemble_train = stacked_train_pred * s + train_pred_lgb *l + train_pred_xgb* x + train_pred_rf * r
ensemble_test = stacked_pred * s + y_pred_lgb * l + y_pred_xgb * x + y_pred_rf *r

In [54]:
print('MAE score on train data:')  # 1844.5
print(mae_(y_train, ensemble_train))

MAE score on train data:
1880.15109109


In [55]:
## prep submission

In [56]:
y_pred_stacked = pd.DataFrame(stacked_pred)
y_pred_lgb = pd.DataFrame(y_pred_lgb)
y_pred_xgb = pd.DataFrame(y_pred_xgb)
y_pred_rf = pd.DataFrame(y_pred_rf)

y_pred = pd.DataFrame(ensemble_test)

all_y = pd.concat([y_pred, y_pred_stacked, y_pred_lgb, y_pred_xgb, y_pred_rf], axis=1)
all_y.columns=['Final', 'Stacked', 'lgb','xgb','rf']
all_y.to_csv('all_y.csv')
all_y.sample(10)

,Final,Stacked,lgb,xgb,rf
120388,484.410808,438.795439,970.702957,590.951355,279.960000
51275,2240.387372,2239.198248,2338.713982,2174.690430,2227.230000
111641,11965.037072,11647.435709,11661.717566,12031.022461,14174.320000
58229,2455.220179,2446.832552,2454.467736,2512.597900,2457.050000
99385,5123.637737,5018.913347,5134.420590,5240.967285,5674.580000
55218,2271.773471,2302.171332,2198.285267,2278.746582,2132.120000
73506,2932.715259,2992.646601,2880.384693,2830.807373,2686.793333
13453,3260.259054,3174.580564,3260.646332,3147.298828,3907.860000
85251,2427.768967,2412.626015,2459.652954,2306.951904,2607.880000
87877,3418.715622,3451.347905,3340.367921,3443.210693,3252.890000


In [57]:
all_y.describe()

,Final,Stacked,lgb,xgb,rf
count,140510.000000,140510.000000,140510.000000,140510.000000,140510.000000
mean,4478.651385,4511.193601,4235.111060,4510.043457,4442.253795
std,4660.780665,4838.799243,3279.690252,4767.360352,4906.939107
min,-246.261616,-1364.954249,202.823529,-3360.771240,0.000000
25%,2182.402503,2172.779360,2239.544298,2169.208801,2048.812500
50%,3017.702271,3000.576547,3023.003088,3001.991455,3028.490000
75%,4962.074172,4939.340710,4971.169514,5017.014526,4951.820000
max,221116.105273,263485.631337,15084.806161,158861.296875,176658.190000


In [58]:
# Negative -> 0 
b = all_y[all_y['Final']<0].shape[0]
a = b/all_y.shape[0]*100
print(("negative prediction (#): %2d"%b))
print("negative prediction (Percent): %2f"%a)

negative prediction (#):  3
negative prediction (Percent): 0.002135


In [59]:
# condition1 = all_y['Final']<0
# all_y['Final'].loc[condition1] = 0
# all_y.describe()

In [60]:
print(df_testing.shape)
print(test_ID.shape)
print(y_pred.shape)

(140510, 2)
(140510,)
(140510, 1)


In [40]:
df = pd.DataFrame(test_ID).reset_index()
# df['Next_Premium'] = all_y['Final']
df['Next_Premium'] = y_pred_lgb
df = df_testing.merge(df, how = 'left', on = 'Policy_Number', suffixes=('_x','_y'))
df=df.drop(['index','Next_Premium_x'], axis = 1)
df.columns = df_testing.columns
df.head()

,Policy_Number,Next_Premium
0,55789b8f86893761c9aa9e7bf17938e737decc68,1
1,b6df13a3384528ba6339c52b4fff7c149de68011,1
2,e112d926103147bcdcb6dab201b736185a3e2520,0
3,aa346fa4b1931d1c7a55f8e1bca40b0927dd65ac,1
4,39c4d5daaa791676ec5559c9066d7e8e8dfc51d7,1


In [44]:
df_lgb = pd.read_csv('testing-set_lgb.csv')
print(df_lgb.shape)
df_lgb.head()

(140510, 2)


,Policy_Number,Next_Premium
0,55789b8f86893761c9aa9e7bf17938e737decc68,2951.893834
1,b6df13a3384528ba6339c52b4fff7c149de68011,1718.870132
2,e112d926103147bcdcb6dab201b736185a3e2520,3056.296986
3,aa346fa4b1931d1c7a55f8e1bca40b0927dd65ac,6113.368185
4,39c4d5daaa791676ec5559c9066d7e8e8dfc51d7,2597.848869


In [51]:
df_final = df.merge(df_lgb, on = 'Policy_Number')
print(df_final.shape)
df_final.head()

(140510, 3)


,Policy_Number,Next_Premium_x,Next_Premium_y
0,55789b8f86893761c9aa9e7bf17938e737decc68,1,2951.893834
1,b6df13a3384528ba6339c52b4fff7c149de68011,1,1718.870132
2,e112d926103147bcdcb6dab201b736185a3e2520,0,3056.296986
3,aa346fa4b1931d1c7a55f8e1bca40b0927dd65ac,1,6113.368185
4,39c4d5daaa791676ec5559c9066d7e8e8dfc51d7,1,2597.848869


In [52]:
df_final['Next_Premium'] = df_final.apply(lambda row: row['Next_Premium_y'] if row['Next_Premium_x']==1 else 0, axis = 1 )
df_final.head()

,Policy_Number,Next_Premium_x,Next_Premium_y,Next_Premium
0,55789b8f86893761c9aa9e7bf17938e737decc68,1,2951.893834,2951.893834
1,b6df13a3384528ba6339c52b4fff7c149de68011,1,1718.870132,1718.870132
2,e112d926103147bcdcb6dab201b736185a3e2520,0,3056.296986,0.000000
3,aa346fa4b1931d1c7a55f8e1bca40b0927dd65ac,1,6113.368185,6113.368185
4,39c4d5daaa791676ec5559c9066d7e8e8dfc51d7,1,2597.848869,2597.848869


In [53]:
df_final = df_final.drop(['Next_Premium_x', 'Next_Premium_y'], axis =1 )
df_final.to_csv('testing-set.csv',index = False) 
df_final.head()

,Policy_Number,Next_Premium
0,55789b8f86893761c9aa9e7bf17938e737decc68,2951.893834
1,b6df13a3384528ba6339c52b4fff7c149de68011,1718.870132
2,e112d926103147bcdcb6dab201b736185a3e2520,0.000000
3,aa346fa4b1931d1c7a55f8e1bca40b0927dd65ac,6113.368185
4,39c4d5daaa791676ec5559c9066d7e8e8dfc51d7,2597.848869


In [54]:
df_final.describescribe()

,Next_Premium
count,140510.000000
mean,4080.800926
std,3382.090936
min,0.000000
25%,2131.202149
50%,2951.893834
75%,4879.284834
max,15084.806161


In [62]:
#save to csv
df.to_csv('testing-set.csv',index = False) 

In [63]:
df = pd.DataFrame(test_ID).reset_index()
# df['Next_Premium'] = all_y['Final']
df['Next_Premium'] = y_pred_rf
df = df_testing.merge(df, how = 'left', on = 'Policy_Number', suffixes=('_x','_y'))
df=df.drop(['index','Next_Premium_x'], axis = 1)
df.columns = df_testing.columns
#save to csv
df.to_csv('testing-set_rf.csv',index = False) 

In [64]:
df = pd.DataFrame(test_ID).reset_index()
# df['Next_Premium'] = all_y['Final']
df['Next_Premium'] = y_pred_lgb
df = df_testing.merge(df, how = 'left', on = 'Policy_Number', suffixes=('_x','_y'))
df=df.drop(['index','Next_Premium_x'], axis = 1)
df.columns = df_testing.columns
#save to csv
df.to_csv('testing-set_lgb.csv',index = False) 

In [65]:
df = pd.DataFrame(test_ID).reset_index()
# df['Next_Premium'] = all_y['Final']
df['Next_Premium'] = y_pred
df = df_testing.merge(df, how = 'left', on = 'Policy_Number', suffixes=('_x','_y'))
df=df.drop(['index','Next_Premium_x'], axis = 1)
df.columns = df_testing.columns
#save to csv
df.to_csv('testing-set_en.csv',index = False) 